Trying to understand scaling laws by [chinchilla]() using [nanogpt (scaling_laws.ipynb)]()

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import functional as F
import pandas as pd
import scipy
%matplotlib inline

In [3]:
def gpt_params(seq_len, vocab_size, d_model, num_heads, num_layers):
    """ Given a GPT calculate total number of parameters """
    ffw_size = 4*d_model # Represent the intermediate layer size in MLP. in GPT the number of intermediate features is always 4*d_model.
    # token and position embeddings
    embeddings = d_model * vocab_size + d_model * seq_len
    # transformer blocks
    attention = 3*d_model**2 + 3*d_model # weights and biases
    attproj = d_model**2 + d_model
    ffw = d_model*(ffw_size) + ffw_size
    ffwproj = ffw_size*d_model + d_model
    layernorms = 2*2*d_model
    # dense
    ln_f = 2*d_model
    dense = d_model*vocab_size # note: no bias here
    # note: embeddings are not included in the param count!
    total_params = num_layers*(attention + attproj + ffw + ffwproj + layernorms) + ln_f + dense
    return total_params

# Explanation

A model's parameter size can simply be derived by running `.parameters()` to recieve the parameter count. However, intuitively let's understand, how the GPT parameters is calculated.

`embedding = d_model * vocab_size + d_model * seq_len`
embedding represents the total number of parameters used in the embedding layer. The embedding layer is made of two parts _token embedding_ and the _positional embedding_. The size of the weights used in the token embedding is `(number of input channels (d_model), vocab_size)`

since 
```
embedding layer    =  Train Data     @ Transpose(Weight)
(B, T, vocab_size) = (B, vocab_size) @ (T, vocab_size)  
```

As you can see, after the training data is hot encoded it has a size of `B x vocab_size`. So based on this the Weight dimensions is `Size(vocab_size, T)`. where `T` represents the number of channels (aka. embedding output dimension). So the total number of parameters in the weight is `vocab_size x T`.

Similarly in the positional embedding, the weight dimension is dictated by the training data.

```
embedding layer    =  Train Data  @ Transpose(Weight)
(B, T, vocab_size) = (B, seq_len) @ (T, seq_len)  
```

Positional embedding is simply done by numbering the input within a fixed context window. This context window size is the `seq_len`(or sequence length).
Based on this, the weight that embeds this data is of the `Size(seq_len, T)`. So making the total embedding of the positional embedding `seq_len x T`.

The total number of parameters used in the embedding later is the sum of these two layers since the values are summed before passing through the next layer.

```
attention = 3*d_model**2 + 3*d_model
```

You've come across __Quadratic Scaling__ or __Squared attention__. Its the thought that when double the length of the input sequence, the computational cost associated with the attention mechanism increases by a factor of four because computational cost of attention in a transformer model scales quadratically with the sequence length.

How does attention work?
```python
k = []
for i in keys:
    ki = []
    for j in values:
        for k in j:
            ki[k] += j


def attention(query, keys, values):
    # Q, K, V  ->  Size(T, 3*C)
    # Initialize attention weights
    attention_weights = [] # (T, T)
    
    # Calculate attention scores
    for key in keys:
        score = dot_product(query, key) # (T, T)
        attention_weights.append(score)
    
    # Normalize attention weights using softmax
    attention_weights = softmax(attention_weights) # (T, T)
    
    # Calculate weighted sum of values
    context_vector = [0] * len(values[0]) #(C,)
    for i in range(len(values)):
        for j in range(len(values[0])):
            context_vector[j] += attention_weights[i] * values[i][j]
    
    return context_vector
```


In [4]:
B, T, C = 2, 4, 2

In [72]:
# first implementation
np.random.seed(10)
x = np.linspace(1, 16, 16).reshape(B, T, C) 
# x /= B*T*C
sol_1 = np.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        x_prev = x[b, :t+1] #(t, C)
        sol_1[b, t] = np.mean(x_prev, 0)
        

sol_1

array([[[ 1.,  2.],
        [ 2.,  3.],
        [ 3.,  4.],
        [ 4.,  5.]],

       [[ 9., 10.],
        [10., 11.],
        [11., 12.],
        [12., 13.]]])

In [81]:
# second implementation

wei = np.tril(np.ones((B, T, T)))
wei = wei /wei.sum(2, keepdims=True)
sol_2 = wei @ x
print(wei)
assert (sol_1 == sol_2).all()
sol_2

[[[1.         0.         0.         0.        ]
  [0.5        0.5        0.         0.        ]
  [0.33333333 0.33333333 0.33333333 0.        ]
  [0.25       0.25       0.25       0.25      ]]

 [[1.         0.         0.         0.        ]
  [0.5        0.5        0.         0.        ]
  [0.33333333 0.33333333 0.33333333 0.        ]
  [0.25       0.25       0.25       0.25      ]]]


array([[[ 1.,  2.],
        [ 2.,  3.],
        [ 3.,  4.],
        [ 4.,  5.]],

       [[ 9., 10.],
        [10., 11.],
        [11., 12.],
        [12., 13.]]])

In [82]:
# third implementation
softmax = lambda v: np.exp(v) / np.exp(v).sum(2, keepdims=True) # hmm, only finding the softmax along the T dimension works. So batch should be ignored??

c = np.tril(np.ones((B, T, T)))
sol_3 = np.zeros((B, T, T))
masked_weight = np.ma.masked_array(sol_3, mask=(c==0))
filled_weight = np.ma.filled(masked_weight, fill_value=-np.inf) # fill all zeros to with -Inf
wei = softmax(filled_weight) # NOTE: WHY 10???
print(wei)
sol_3 = wei @ x

assert (sol_1 == sol_3).all()
sol_3

[[[1.         0.         0.         0.        ]
  [0.5        0.5        0.         0.        ]
  [0.33333333 0.33333333 0.33333333 0.        ]
  [0.25       0.25       0.25       0.25      ]]

 [[1.         0.         0.         0.        ]
  [0.5        0.5        0.         0.        ]
  [0.33333333 0.33333333 0.33333333 0.        ]
  [0.25       0.25       0.25       0.25      ]]]


array([[[ 1.,  2.],
        [ 2.,  3.],
        [ 3.,  4.],
        [ 4.,  5.]],

       [[ 9., 10.],
        [10., 11.],
        [11., 12.],
        [12., 13.]]])

In [85]:
# forth implementation

softmax = lambda v: np.exp(v) / np.exp(v).sum(2, keepdims=True) # hmm, only finding the softmax along the T dimension works. So batch should be ignored??
k = np.ones((B, T, C))
q = np.ones((B, T, C)) 
v = x #np.ones((B, T, C))

d_n = 16
attn = q @ np.transpose(k, (0, 2, 1)) / C # (T, T)

c = np.tril(np.ones((B, T, T)))
masked_weight = np.ma.masked_array(attn, mask=(c==0))
filled_weight = np.ma.filled(masked_weight, fill_value=-np.inf) # fill all zeros to with -Inf
wei = softmax(filled_weight) # NOTE: WHY 10???
print(wei)
sol_4 = wei @ v

assert (sol_1 == sol_4).all()
sol_4

[[[1.         0.         0.         0.        ]
  [0.5        0.5        0.         0.        ]
  [0.33333333 0.33333333 0.33333333 0.        ]
  [0.25       0.25       0.25       0.25      ]]

 [[1.         0.         0.         0.        ]
  [0.5        0.5        0.         0.        ]
  [0.33333333 0.33333333 0.33333333 0.        ]
  [0.25       0.25       0.25       0.25      ]]]


array([[[ 1.,  2.],
        [ 2.,  3.],
        [ 3.,  4.],
        [ 4.,  5.]],

       [[ 9., 10.],
        [10., 11.],
        [11., 12.],
        [12., 13.]]])

ohh!! Initially thought Q, K, V was inspired by a theory with a similar trifecta. However that doesn't seem to be the case. 
It seems as though, it was inspired by information retrival or something of that sought but nothing pertaining numerical analysis.

In [86]:
np.linspace(1, 4, 4).reshape(2, 2) @ np.ones((2, 2))

array([[3., 3.],
       [7., 7.]])

In [104]:
# fifth implementation
query = lambda v: v @ (np.ones((B, C, 3*C)) / C)
key = lambda v: v @ (np.ones((B, C, 3*C)) / C)
value = lambda v: v @ (np.ones((B, C, 3*C)) / C)

softmax = lambda v: np.exp(v) / np.exp(v).sum(2, keepdims=True) # hmm, only finding the softmax along the T dimension works. So batch should be ignored??
k = np.ones((B, T, C))
q = np.ones((B, T, C)) 
v = x #np.ones((B, T, C))
# print(key(k)[0], query(q)[0], value(v)[0])
d_n = 16
attn = (query(q) @ np.transpose(key(k), (0, 2, 1))) / (3*C) # (T, T)

# print(attn, attn.shape)
c = np.tril(np.ones((B, T, T)))
masked_weight = np.ma.masked_array(attn, mask=(c==0))
filled_weight = np.ma.filled(masked_weight, fill_value=-np.inf) # fill all zeros to with -Inf
wei = softmax(filled_weight) # NOTE: WHY 10???
print(wei)
sol_5 = wei @ value(v)

assert np.allclose(query(sol_1), sol_5)
sol_5

[[[1.         0.         0.         0.        ]
  [0.5        0.5        0.         0.        ]
  [0.33333333 0.33333333 0.33333333 0.        ]
  [0.25       0.25       0.25       0.25      ]]

 [[1.         0.         0.         0.        ]
  [0.5        0.5        0.         0.        ]
  [0.33333333 0.33333333 0.33333333 0.        ]
  [0.25       0.25       0.25       0.25      ]]]


array([[[ 1.5,  1.5,  1.5,  1.5,  1.5,  1.5],
        [ 2.5,  2.5,  2.5,  2.5,  2.5,  2.5],
        [ 3.5,  3.5,  3.5,  3.5,  3.5,  3.5],
        [ 4.5,  4.5,  4.5,  4.5,  4.5,  4.5]],

       [[ 9.5,  9.5,  9.5,  9.5,  9.5,  9.5],
        [10.5, 10.5, 10.5, 10.5, 10.5, 10.5],
        [11.5, 11.5, 11.5, 11.5, 11.5, 11.5],
        [12.5, 12.5, 12.5, 12.5, 12.5, 12.5]]])